In [1]:
! pip install modelscope==1.18.0
! pip install transformers==4.44.2
! pip install sentencepiece==0.2.0
! pip install accelerate==0.34.2
! pip install datasets==2.20.0
! pip install peft==0.11.1

In [2]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer,AutoModelForCausalLM,DataCollatorForSeq2Seq, TrainingArguments, Trainer,GenerationConfig
import torch

2025-10-27 07:22:55.721091: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761549775.744454     282 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761549775.751650     282 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# 数据地址  https://github.com/datawhalechina/self-llm/blob/master/dataset/huanhuan.json/

In [4]:
df = pd.read_json("/kaggle/input/huanyuan-sft/huanhuan.json")
ds = Dataset.from_pandas(df)

ds

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 3729
})

In [5]:
ds[:3]

{'instruction': ['小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——',
  '这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱们府里跑。',
  '嬛妹妹，刚刚我去府上请脉，听甄伯母说你来这里进香了。'],
 'input': ['', '', ''],
 'output': ['嘘——都说许愿说破是不灵的。', '你们俩话太多了，我该和温太医要一剂药，好好治治你们。', '出来走走，也是散心。']}

In [6]:
#下载，加载模型和分词器
! modelscope download --model Qwen/Qwen2.5-1.5B-Instruct --local_dir 'Qwen/Qwen1.5-1.5B-Instruct'

In [7]:
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-1.5B-Instruct", torch_dtype=torch.bfloat16  )
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-1.5B-Instruct")

In [8]:
tokenizer

Qwen2TokenizerFast(name_or_path='Qwen/Qwen1.5-1.5B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_word=Fals

In [9]:
def process_fun(example):
  MAX_LENGTH=384
  input_ids, attention_mask, labels = [], [], []
  instruction =tokenizer(f"<|im_start|>system\n现在你要扮演皇帝身边的女人-甄嬛<|im_end|>\n<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)
  response = tokenizer(f"{example['output']}", add_special_tokens=False)
  input_ids = instruction["input_ids"] + response["input_ids"]+[tokenizer.eos_token_id]
  attention_mask = instruction["attention_mask"] + response["attention_mask"] +[1]
  labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eos_token_id]
  #超出最大长度进行截断处理
  if len(input_ids) > MAX_LENGTH:
    input_ids = input_ids[:MAX_LENGTH]
    attention_mask = attention_mask[:MAX_LENGTH]
    labels = labels[:MAX_LENGTH]

  return {
      "input_ids": input_ids,
      "attention_mask": attention_mask,
      "labels": labels
  }

In [10]:
tokenized_id = ds.map(process_fun,remove_columns=ds.column_names)
tokenized_id

Map:   0%|          | 0/3729 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3729
})

In [11]:
tokenizer.decode(tokenized_id[0]["input_ids"])

'<|im_start|>system\n现在你要扮演皇帝身边的女人-甄嬛<|im_end|>\n<|im_start|>user\n小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——<|im_end|>\n<|im_start|>assistant\n嘘——都说许愿说破是不灵的。<|im_end|>'

In [12]:
tokenizer.decode(list(filter(lambda x:x!=-100,tokenized_id[1]["labels"])))

'你们俩话太多了，我该和温太医要一剂药，好好治治你们。<|im_end|>'

In [13]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qw

In [14]:
model.enable_input_require_grads()  #开启梯度检查点

In [15]:
from peft import LoraConfig,TaskType,get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_attn","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    inference_mode=False, #训练模式
    r = 8,
    lora_alpha = 32,
    lora_dropout = 0
)

model = get_peft_model(model,config)

model.print_trainable_parameters()

trainable params: 8,544,256 || all params: 1,552,258,560 || trainable%: 0.5504


In [16]:
args = TrainingArguments(
    output_dir = "/kaggle/working/output/Qwen2.5_instruct_lora",
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4,
    learning_rate = 1e-4,
    num_train_epochs = 3,
    save_steps = 100,
    save_on_each_node = True,
    logging_steps = 10,
    gradient_checkpointing = True
)

In [17]:
import wandb
wandb.login(key="8e46232494a07419d773e0ffb1a25be17a1fa86f")  # 替换为实际的API密钥

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

True

In [18]:
trainer = Trainer(
    model = model,
    args  =args,
    train_dataset = tokenized_id,
    data_collator = DataCollatorForSeq2Seq(tokenizer,padding=True),
)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.648800
20,3.450700
30,3.429600
40,3.270400
50,3.262800
60,3.286900
70,3.224400
80,3.345700
90,3.343300
100,3.286700


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=699, training_loss=2.984894224502498, metrics={'train_runtime': 3078.8425, 'train_samples_per_second': 3.634, 'train_steps_per_second': 0.227, 'total_flos': 8511262565870592.0, 'train_loss': 2.984894224502498, 'epoch': 2.996784565916399})

In [31]:
#测试不合并权重模型输出
# 合并权重进行推理
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = 'Qwen/Qwen2.5-1.5B-Instruct'

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

prompt = "你是谁？"
inputs = tokenizer.apply_chat_template([{"role": "user", "content": "假设你是皇帝身边的女人--甄嬛。"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 2048, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

我是《甄嬛传》中的虚构角色，甄嬛。在电视剧中，我是一个聪明、机智且具有复杂性格的女性角色。我的故事围绕着宫廷斗争和权力争夺展开，展现了古代中国社会的政治、文化以及人性的多面性。如果您有任何关于甄嬛或《甄嬛传》的问题，请随时告诉我，我会尽力提供帮助。


In [35]:
# 合并权重进行推理
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = 'Qwen/Qwen2.5-1.5B-Instruct'
lora_path = '/kaggle/working/output/Qwen2.5_instruct_lora/checkpoint-600' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

prompt = "你是谁？"
# prompt = "皇后说你僭越了，该当如何"
prompt = "嬛嬛你怎么了，朕替你打抱不平！"
inputs = tokenizer.apply_chat_template([{"role": "user", "content": "假设你是皇帝身边的女人--甄嬛。"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 2048, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

臣妾没事，只是有些累了。
